In [2]:
pip install pyecharts

Defaulting to user installation because normal site-packages is not writeable
     ------------------------------------ 147.1/147.1 kB 977.6 kB/s eta 0:00:00
     ---------------------------------------- 75.5/75.5 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import datetime
import jieba
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['font.sans-serif'] = ['SimHei'] 
matplotlib.rcParams['font.family']='sans-serif'
matplotlib.rcParams['axes.unicode_minus'] = False
from chinese_calendar import is_workday
from pyecharts.charts import Bar

In [8]:
login=pd.read_csv('login(ANSI编码).csv',encoding='ANSI')
login.info()
# 没有缺失值

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387144 entries, 0 to 387143
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   user_id      387144 non-null  object
 1   login_time   387144 non-null  object
 2   login_place  387144 non-null  object
dtypes: object(3)
memory usage: 8.9+ MB


In [9]:
#一天内重复登录的行为我们将视为一次登录即可，以减少数据量
 
# 方法一
# data1=login['login_time'].str.split(' ',expand=True)[0]
# login=pd.concat([login,data1],axis=1)
# login.rename(columns={0:'日期'},inplace=True)
# del login['login_time']
# login
 
# 方法二
def str_datetime(df,column_name='login_time'):
    df[column_name]=pd.to_datetime(df[column_name])
    df[column_name]=df[column_name].apply(lambda x:x.strftime('%Y-%m-%d'))
    return df
 
login=str_datetime(login)
# 将同一日期用户重复登陆行为删除
login=login.drop_duplicates(subset=['user_id'])
login

,user_id,login_time,login_place
0,用户3,2018-09-06,中国广东广州
1661,用户4,2018-09-10,中国广东广州
1888,用户5,2018-09-10,中国广东广州
2371,用户6,2018-09-11,中国广东广州
2372,用户7,2018-09-11,中国广东广州
...,...,...,...
387138,用户44246,2020-06-17,中国广东肇庆
387139,用户44247,2020-06-18,中国湖北武汉
387141,用户44248,2020-06-18,中国天津
387142,用户44249,2020-06-18,中国北京


In [10]:

# 以最近的时间为基准，计算出用户每一次登录距离现在的时间
login['时间差1']=pd.to_datetime(login['login_time']).max()-pd.to_datetime(login['login_time'])
login

,user_id,login_time,login_place,时间差1
0,用户3,2018-09-06,中国广东广州,651 days
1661,用户4,2018-09-10,中国广东广州,647 days
1888,用户5,2018-09-10,中国广东广州,647 days
2371,用户6,2018-09-11,中国广东广州,646 days
2372,用户7,2018-09-11,中国广东广州,646 days
...,...,...,...,...
387138,用户44246,2020-06-17,中国广东肇庆,1 days
387139,用户44247,2020-06-18,中国湖北武汉,0 days
387141,用户44248,2020-06-18,中国天津,0 days
387142,用户44249,2020-06-18,中国北京,0 days


In [11]:

login=login.reset_index()
del login['index']

In [ ]:
for i in range(login.shape[0]):
    if login.loc[i,'login_place'][0:2]=='中国':
        login.loc[i,'国家']='中国'
        if '黑龙江' in login.loc[i,'login_place']:
            login.loc[i,'省份']='黑龙江'
            if len(login.loc[i,'login_place'])>5:
                login.loc[i,'地区']=login.loc[i,'login_place'][5:]
            else:pass
        if '新疆维吾尔' in login.loc[i,'login_place']:
            login.loc[i,'省份']='新疆维吾尔'
            if len(login.loc[i,'login_place'])>7:
                login.loc[i,'地区']=login.loc[i,'login_place'][7:]
            else:pass
        if '内蒙古' in login.loc[i,'login_place']:
            login.loc[i,'省份']='内蒙古'
            if len(login.loc[i,'login_place'])>5:
                login.loc[i,'地区']=login.loc[i,'login_place'][5:]
            else:pass
        else:
            login.loc[i,'省份']=login.loc[i,'login_place'][2:4]
            login.loc[i,'地区']=login.loc[i,'login_place'][4:]   
    else:
        li=[word for word in jieba.cut(login.iloc[i,2])]
        if len(li)==2:
            login.loc[i,'国家']=li[0]
            login.loc[i,'省份']=li[1]
        else:
            login.loc[i,'国家']=li[0]
    if i%10000==0:
        print(f'{round(i*100/(int(login.shape[0])),2)}%')

0.0%
26.08%
52.17%
78.25%


In [7]:
login.to_excel('用户地区切割.xlsx')


In [8]:
# 重新读取已经切分好城市的数据
login =pd.read_excel('用户地区切割.xlsx',index_col=0)
stu_info=pd.read_csv('study_information(ANSI编码).csv',encoding='ANSI')
stu_info=stu_info.drop_duplicates()
stu_info.info()
#没有完全重复数据,但是存在价格缺失数据

stu_info[stu_info.price.isnull()].course_id.value_counts()
 
stu_course=stu_info.groupby(['course_id']).agg({'price':['max','min']})
stu_course[(stu_course['price']['max']-stu_course['price']['min'])!=0]
#不存在差异定价,且51和96课程价格为nan，其中51和96课程共计4238条数据，暂时不做处理
# 将进度转化成为数值型，从而便于计算
stu_info['learn_process']=stu_info['learn_process'].apply(lambda x:int(x.split('width:')[-1].split('%')[0]))
# 对加入时间进行相同操作
stu_info=str_datetime(stu_info,column_name='course_join_time')
stu_info
users=pd.read_csv('new_users(ANSI编码).csv',encoding='ANSI')
users

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194974 entries, 0 to 194973
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           194974 non-null  object 
 1   course_id         194974 non-null  object 
 2   course_join_time  194974 non-null  object 
 3   learn_process     194974 non-null  object 
 4   price             190736 non-null  float64
dtypes: float64(1), object(4)
memory usage: 8.9+ MB


In [9]:
users=users[~users.user_id.isnull()]
users.info()
 
# 将是否填写学校信息设置成为0、1变量，作为后续可能有用的信息
users['是否填写学校信息']=users['school']
users.是否填写学校信息[~users['是否填写学校信息'].isnull()]=1
users.是否填写学校信息[users['是否填写学校信息'].isnull()]=0
users.是否填写学校信息.value_counts()


C:\Users\ttyai\AppData\Local\Temp\ipykernel_21440\2403811793.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.是否填写学校信息[~users['是否填写学校信息'].isnull()]=1
C:\Users\ttyai\AppData\Local\Temp\ipykernel_21440\2403811793.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users.是否填写学校信息[users['是否填写学校信息'].isnull()]=0


0    33347
1    10569
Name: 是否填写学校信息, dtype: int64

In [11]:
keys=login.groupby('user_id').login_time.max().index.tolist()
values=login.groupby('user_id').login_time.max().values.tolist()
login_time={}
for i in range(len(keys)):
    login_time[keys[i]]=values[i]
    
u_2=users[users.recently_logged!='--']
u_1=users[users.recently_logged=='--']
for i in range(u_1.shape[0]):
    if u_1.iloc[i,0] in login_time.keys():
        u_1.iloc[i,2]=pd.to_datetime(login_time[u_1.iloc[i,0]])
    else:
        if pd.to_datetime(u_1.iloc[i,1])+datetime.timedelta(days  = int(u_1.iloc[i,5])/480 )>pd.to_datetime('2020-06-18'):
            u_1.iloc[i,2]=pd.to_datetime('2020-06-18')
            print('修改时间为最新时间')
        else:
            u_1.iloc[i,2]=pd.to_datetime(u_1.iloc[i,1])+datetime.timedelta(days  = int(u_1.iloc[i,5])/480 )        
users=pd.concat([u_1,u_2])

users['recently_logged'].isna().sum()
 
# 还是以最近的时间为基准，计算出用户登陆注册行为的时间差
users = users[~users['recently_logged'].isnull()]
str_datetime(users,column_name='register_time')
str_datetime(users,column_name='recently_logged')
users['register_logged_time']=pd.to_datetime(users['recently_logged'])-pd.to_datetime(users['register_time'])
users['register_now_time']=pd.to_datetime(users['register_time']).max()-pd.to_datetime(users['register_time'])
users['logged_now_time']=pd.to_datetime(users['recently_logged']).max()-pd.to_datetime(users['register_time'])
users
 
# 现在加入的班级的数量
users['number_of_classes_now']=users['number_of_classes_join']-users['number_of_classes_out']
users

,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now
0,0,用户44251,2020-06-18,2020-06-18,0,0,41.25,NaN,0,0 days,0 days,0 days,0
1,1,用户44250,2020-06-18,2020-06-18,0,0,0,NaN,0,0 days,0 days,0 days,0
2,2,用户44249,2020-06-18,2020-06-18,0,0,16.22,NaN,0,0 days,0 days,0 days,0
3,3,用户44248,2020-06-18,2020-06-18,0,0,0,NaN,0,0 days,0 days,0 days,0
4,4,用户44247,2020-06-18,2020-06-18,0,0,1.8,NaN,0,0 days,0 days,0 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43911,43978,用户6,2018-09-11,2018-09-11,0,0,0,NaN,0,0 days,646 days,646 days,0
43912,43979,用户5,2018-09-10,2020-06-15,9,5,"2,116.15",NaN,0,644 days,647 days,647 days,4
43913,43980,用户4,2018-09-10,2020-06-05,4,0,341.2,NaN,0,634 days,647 days,647 days,4
43914,43981,用户3,2018-09-04,2020-06-18,2,1,370.35,NaN,0,653 days,653 days,653 days,1


In [12]:
def nx_data(df=stu_info,group_name=['course_id','user_id']):
    # 得到共现字典
    user_dic={}  
    stu_info_data=df.groupby(group_name)['course_id'].count().unstack()
    column=stu_info_data.columns.tolist()
    for i in range(stu_info_data.shape[0]):
        user_dic[column[i]]=stu_info_data[stu_info_data[column[i]]==1].index.tolist()
        
    #构造共现矩阵
    course_name=list(set(df['course_id'].values.tolist()))
    course_data=pd.DataFrame(data=np.zeros(shape=(len(course_name),len(course_name))),index=course_name,columns=course_name)
    for value in user_dic.values():
        if len(value)==1:
            pass
        else:
            for i in range(len(value)):
                for j in range(i+1,len(value)):
                    course_data.loc[value[i],value[j]]+=1
    return (user_dic,course_data)

user_dic,course_data=nx_data()

for i,key in enumerate(user_dic.keys()):
    users.loc[i,'选课数量']=len(user_dic[key])
users

,Unnamed: 0,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量
0,0.0,用户44251,2020-06-18,2020-06-18,0.0,0.0,41.25,NaN,0,0 days,0 days,0 days,0.0,2.0
1,1.0,用户44250,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,0 days,0 days,0.0,1.0
2,2.0,用户44249,2020-06-18,2020-06-18,0.0,0.0,16.22,NaN,0,0 days,0 days,0 days,0.0,1.0
3,3.0,用户44248,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,0 days,0 days,0.0,1.0
4,4.0,用户44247,2020-06-18,2020-06-18,0.0,0.0,1.8,NaN,0,0 days,0 days,0 days,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43913,43980.0,用户4,2018-09-10,2020-06-05,4.0,0.0,341.2,NaN,0,634 days,647 days,647 days,4.0,NaN
43914,43981.0,用户3,2018-09-04,2020-06-18,2.0,1.0,370.35,NaN,0,653 days,653 days,653 days,1.0,NaN
43915,43982.0,用户1,2018-09-03,2018-11-04,0.0,0.0,0,NaN,0,62 days,654 days,654 days,0.0,NaN
37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,3.0


In [13]:
login_1=login.sort_values(by=['user_id','时间差1'])
login_del=login_1.user_id.drop_duplicates()
login_diff=login.iloc[list(login_del.index),:]
users_all=pd.merge(users,login_diff,how='left')
users_all.to_excel('全部信息.xlsx')
users_all=users_all.reset_index()
users_all=users_all.drop(columns=['index'])
users_all
users_all = users_all.drop(labels='Unnamed: 0', axis = 1)
users_all

,user_id,register_time,recently_logged,number_of_classes_join,number_of_classes_out,learn_time,school,是否填写学校信息,register_logged_time,register_now_time,logged_now_time,number_of_classes_now,选课数量,login_time,login_place,时间差1,国家,省份,地区
0,用户44251,2020-06-18,2020-06-18,0.0,0.0,41.25,NaN,0,0 days,0 days,0 days,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,用户44250,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,0 days,0 days,0.0,1.0,2020-06-18,中国江西南昌,0.0,中国,江西,南昌
2,用户44249,2020-06-18,2020-06-18,0.0,0.0,16.22,NaN,0,0 days,0 days,0 days,0.0,1.0,2020-06-18,中国北京,0.0,中国,北京,NaN
3,用户44248,2020-06-18,2020-06-18,0.0,0.0,0,NaN,0,0 days,0 days,0 days,0.0,1.0,2020-06-18,中国天津,0.0,中国,天津,NaN
4,用户44247,2020-06-18,2020-06-18,0.0,0.0,1.8,NaN,0,0 days,0 days,0 days,0.0,1.0,2020-06-18,中国湖北武汉,0.0,中国,湖北,武汉
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43721,用户4,2018-09-10,2020-06-05,4.0,0.0,341.2,NaN,0,634 days,647 days,647 days,4.0,NaN,2018-09-10,中国广东广州,647.0,中国,广东,广州
43722,用户3,2018-09-04,2020-06-18,2.0,1.0,370.35,NaN,0,653 days,653 days,653 days,1.0,NaN,2018-09-06,中国广东广州,651.0,中国,广东,广州
43723,用户1,2018-09-03,2018-11-04,0.0,0.0,0,NaN,0,62 days,654 days,654 days,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
